<a href="https://colab.research.google.com/github/h4x4d/narod_parser/blob/main/narod_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prequesties**:

### **Install selenium && other dependencies:**

In [2]:
%%capture
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

pip install selenium

pip install html2text

### **Start selenium browser:**

In [2]:
from selenium import webdriver


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(options=chrome_options)

### **Database**


In [6]:
import sqlite3

conn = sqlite3.connect('data.db')
cur = conn.cursor()

In [7]:
cur.execute('''
CREATE TABLE sites (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  title TEXT,
  url TEXT,
  pure_html TEXT,
  md_text TEXT,
  plain_text TEXT,
  has_creenshot BOOLEAN,
  screenshot_url TEXT,
  screenshot_thumbnail BLOB
)
''')

## **Get site data**:

In [8]:
import requests
import re
from urllib.parse import urljoin
import os
import io
import shutil
from PIL import Image
import html2text
from bs4 import BeautifulSoup
from urllib.request import urlopen
from lxml.html import fromstring
import datetime
import time

In [9]:
def make_links_absolute(html):
  absolutize = lambda m: ' src="' + urljoin(sources, m.group(1)) + '"'
  html = re.sub(r' src="([^"]+)"', absolutize, html)
  absolutize2 = lambda m: ' href="' + urljoin(sources, m.group(1)) + '"'
  html = re.sub(r' href="([^"]+)"', absolutize2, html)
  return html

In [10]:
def upload_image(image):

  params = {
    'key': '6d207e02198a847aa98d0a2a901485a5',
    'action': 'upload',
    'format': 'json'
  }
  source = {
      'source': image
  }

  resp = requests.post('https://freeimage.host/api/1/upload', params=params, files=source)

  return resp.json()['image']['url']

In [11]:
def get_plain_text(html):
  soup = BeautifulSoup(html, features="html.parser")
  title = soup.title.string

  for script in soup(["script", "style"]):
      script.extract()

  text = soup.get_text()

  lines = (line.strip() for line in text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  return ('\n'.join(chunk for chunk in chunks if chunk), title)

In [12]:
def save_thumbnail(temp):
  image = Image.open(temp)
  x, y = image.size
  image = image.crop((0, 0, x - 15, y-15))
  image = image.resize((100, 66), Image.Resampling.LANCZOS)
  image.convert('RGB').save('thumb.jpg')

In [13]:
def screenshot(site_url):
  try:
    driver.get(site_url)
  except:
    print('site have not loaded on time, force exit')
  temp = io.BytesIO(driver.get_screenshot_as_png())

  # upload full image
  screen_url = upload_image(temp)

  # save thumbnail
  save_thumbnail(temp)

  return screen_url

In [14]:
%%time

sites = open('Narod.txt')

driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(5)
driver.set_window_size(1024, 768)

time_start = datetime.datetime.now()

for index, site_url in enumerate(sites):
  # variables
  site_url = site_url.strip()
  sources = site_url[:site_url.rfind('/')]
  path = sources[sources.find('://') + 3:].replace('/', '_')

  resp = requests.get(site_url)

  # make all links absolute
  html = make_links_absolute(resp.text)

  # remove ads
  html = html[:html.rfind('<!-- copyright (t2) -->')]

  plain_text, title = get_plain_text(html)


  # take screenshot
  if (index % 10 == 0):
    screen = screenshot(site_url)

    cur.execute('INSERT INTO sites VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (None, title, site_url, html, html2text.html2text(html),
                 plain_text, True, screen,
                 open('thumb.jpg', 'rb').read()))
  else:
    cur.execute('INSERT INTO sites VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (None, title, site_url, html, html2text.html2text(html),
                 plain_text, False, None, None))

  conn.commit()
  print(index, site_url)

  print('time all:', datetime.datetime.now() - time_start, 'average', (datetime.datetime.now() - time_start) / (index + 1))
  print('-------')

0 http://shkolagre.narod.ru/
time all: 0:00:04.611583 average 0:00:04.611592
-------
1 http://franceschool.narod.ru/
time all: 0:00:05.202583 average 0:00:02.601296
-------
2 http://school66pnz.narod.ru/
time all: 0:00:05.692314 average 0:00:01.897441
-------
3 http://site-bsos2.narod.ru/
time all: 0:00:05.933372 average 0:00:01.483345
-------
4 http://www.site-sch354lite.narod.ru/
time all: 0:00:06.372484 average 0:00:01.274499
-------
5 http://borezschool.narod.ru/
time all: 0:00:06.578834 average 0:00:01.096474
-------
6 https://school-number-eight.narod.ru/
time all: 0:00:07.272403 average 0:00:01.038916
-------
7 http://shatkjno.narod.ru/
time all: 0:00:07.712008 average 0:00:00.964002
-------
8 http://verhotorschool.narod.ru/
time all: 0:00:08.489262 average 0:00:00.943252
-------
9 http://kodschool4.narod.ru/
time all: 0:00:09.573658 average 0:00:00.957367
-------
10 http://ars-school4.narod.ru/site_teacher.htm
time all: 0:00:12.230245 average 0:00:01.111841
-------
11 http://sc

## **Using data:**

In [19]:
import pandas as pd

df = pd.read_sql_query("SELECT * from sites", conn)

df.head(20)

,id,title,url,pure_html,md_text,plain_text,has_creenshot,screenshot_url,screenshot_thumbnail
0,1,Школа при Посольстве России в Греции,http://shkolagre.narod.ru/,"<script type=""text/javascript"" src=""http://s20...",Школа при Посольстве России в Греции\n![](http...,Школа при Посольстве России в Греции\nДобро по...,1,https://iili.io/Jnccoss.png,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,2,Школа во Франции,http://franceschool.narod.ru/,"<script type=""text/javascript"" src=""http://s20...",![](http://franceschool.narod.ru/image/logosch...,Школа во Франции\nОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА ПР...,0,None,None
2,3,Персональный сайт - Главная страница,http://school66pnz.narod.ru/,<!DOCTYPE html>\n<html>\n<head>\n<meta http-eq...,![](http://school66pnz.narod.ru/.s/t/908/1.gif...,Персональный сайт - Главная страница\nВерсия д...,0,None,None
3,4,- Сайт школы № 2 г.Бокситогорска,http://site-bsos2.narod.ru/,"<html xmlns=""http://www.w3.org/1999/xhtml"" xml...",\n![](http://site-bsos2.narod.ru/.s/t/111/3....,"- Сайт школы № 2 г.Бокситогорска\nСуббота, 18....",0,None,None
4,5,Средняя общеобразовательная школа №354,http://www.site-sch354lite.narod.ru/,"\n<script type=""text/javascript"" src=""http://s...",** Департамент образования города Москвы**\n\n...,Средняя общеобразовательная школа №354\nДепарт...,0,None,None
5,6,Персональный сайт - МОУ Борецкая СОШ Сараевско...,http://borezschool.narod.ru/,"<html xmlns=""http://www.w3.org/1999/xhtml"" xml...",[**Версия для слабовидящих**](javascript://)\n...,Персональный сайт - МОУ Борецкая СОШ Сараевско...,0,None,None
6,7,Персональный сайт - Главная,https://school-number-eight.narod.ru/,"<html>\n<head>\n<meta http-equiv=""content-type...",Размер шрифта: A AA Изображения Выключить Вклю...,Персональный сайт - Главная\nРазмер шрифта:\nA...,0,None,None
7,8,МБОУ ООШ с.Новое Шаткино - Главная страница,http://shatkjno.narod.ru/,<html>\n<head>\n<title>МБОУ ООШ с.Новое Шаткин...,Приветствую Вас **Гость** | [RSS](http://shatk...,МБОУ ООШ с.Новое Шаткино - Главная страница\nП...,0,None,None
8,9,Персональный сайт - Новости,http://verhotorschool.narod.ru/,<!DOCTYPE html>\n<html>\n<head>\n<meta http-eq...,| МБОУ СОШ с. Верхотор МР Ишимбайский район Р...,Персональный сайт - Новости\nМБОУ СОШ с. Верхо...,0,None,None
9,10,"МБОУ ""Кодинская средняя общеобразовательная шк...",http://kodschool4.narod.ru/,"<script type=""text/javascript"" src=""http://s20...",﻿ ![](images/01.png) \n--- \n[![Дневник.ру](...,"﻿\nМБОУ ""Кодинская средняя общеобразовательная...",0,None,None


## **On exit**:

### **Quit driver:**

In [ ]:
driver.quit()